# Probabilidad de que UC salga campeón en fecha 28

In [1]:
import pandas as pd
import numpy as np
import matplotlib

In [2]:
total_fechas = 30
fecha_analisis = 28
partidos_x_fecha = 8
equipo_analisis = 'U.Católica (Chile)'

In [3]:
df = pd.read_csv('probs.csv', sep = ';', encoding = 'latin1')

In [4]:
df = df.iloc[:-(total_fechas-fecha_analisis)*partidos_x_fecha]

In [5]:
df.tail(3)

,LOCAL,VISITA,GL,EM,GV
221,U.La Calera (Chile),Coquimbo Unido (Chile),0.36,0.33,0.30
222,U.Católica (Chile),Palestino (Chile),0.49,0.32,0.19
223,U.de Chile (Chile),U.de Concepción (Chile),0.42,0.31,0.26


In [6]:
total_partidos = df.shape[0]
total_partidos

224

In [7]:
df_equipos = pd.read_csv('tabla_equipos_actual.csv', sep = ';', encoding = 'latin-1')
equipos = list(df_equipos['EQUIPO'])
equipos

['U.Católica (Chile)',
 'Colo Colo (Chile)',
 'Palestino (Chile)',
 'Audax Italiano (Chile)',
 "O'Higgins (Chile)",
 'Huachipato (Chile)',
 'Coquimbo Unido (Chile)',
 'U.La Calera (Chile)',
 'U.Española (Chile)',
 'Cobresal (Chile)',
 'Curicó Unido (Chile)',
 'Everton (Chile)',
 'Iquique (Chile)',
 'Antofagasta (Chile)',
 'U.de Chile (Chile)',
 'U.de Concepción (Chile)']

In [8]:
def suma_puntos(row):
    indice_local = equipos.index(row['LOCAL'])
    indice_visita = equipos.index(row['VISITA'])
    if row['r_uniform'] <= row['GL']:
        puntajes[indice_local] += 3
    elif row['r_uniform'] >= 1 - row['GV']:
        puntajes[indice_visita] += 3
    else:
        puntajes[indice_local] += 1
        puntajes[indice_visita] += 1

In [9]:
n_iter = 10000
puntajes_guardados = []
posiciones_guardadas = []
for i in range(n_iter):
    puntajes = [0] * len(equipos)
    df['r_uniform'] = np.random.uniform(low=0.0, high=1.0, size=total_partidos)
    df.apply(suma_puntos, axis=1)
    puntajes_guardados.append(puntajes)
    
    s = pd.Series(puntajes)
    posiciones = list(s.rank(method='first', ascending = False))
    posiciones_guardadas.append(posiciones)

In [10]:
puntajes_guardados_df = pd.DataFrame(puntajes_guardados, columns = equipos)

In [11]:
puntajes_guardados_df.tail(3)

,U.Católica (Chile),Colo Colo (Chile),Palestino (Chile),Audax Italiano (Chile),O'Higgins (Chile),Huachipato (Chile),Coquimbo Unido (Chile),U.La Calera (Chile),U.Española (Chile),Cobresal (Chile),Curicó Unido (Chile),Everton (Chile),Iquique (Chile),Antofagasta (Chile),U.de Chile (Chile),U.de Concepción (Chile)
9997,61,46,43,41,41,39,41,38,37,42,30,30,28,31,29,29
9998,59,45,43,41,47,43,38,35,38,37,34,32,28,27,33,24
9999,66,45,46,46,42,43,42,33,34,35,32,31,28,26,30,26


In [12]:
boxplot = puntajes_guardados_df.boxplot(rot=90)

In [13]:
puntajes_guardados_df['second_place'] = puntajes_guardados_df.apply(lambda row: row.nlargest(2).values[-1],axis=1)

In [14]:
puntajes_guardados_df.loc[puntajes_guardados_df[equipo_analisis] > puntajes_guardados_df['second_place'] + 3*(total_fechas - fecha_analisis), equipo_analisis + '_campeon'] = 1
puntajes_guardados_df.loc[puntajes_guardados_df[equipo_analisis] <= puntajes_guardados_df['second_place'] + 3*(total_fechas - fecha_analisis), equipo_analisis + '_campeon'] = 0

In [15]:
sum(puntajes_guardados_df[equipo_analisis + '_campeon'])

9519.0